In [1]:
import pandas as pd 
import numpy as np 
import re
import matplotlib.pyplot as plt
import math
import seaborn as sns

### Loading Data

In [2]:
tDat = pd.read_csv("train.csv")

In [3]:
tDat.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')

### How dispropotionate is the data???

In [4]:
print("Proportion of the fraud: ", round(tDat.FraudResult.value_counts()[1]*100/len(tDat), 2), "%")

Proportion of the fraud:  0.2 %


<p>Highly dispropotionate data. Possible sampling method:</p> 
<ol>
    <li> Negative Undersampling </li>
    <li> SMOTE (Oversampling) </li>
</ol>

In [5]:
def UnderSample(data, neg_mul, target, seed=None):
    """
    data = data you want to undersample, must be pandas dataframe
    neg_mul = x in x*number of positive sample
    target = target variable column name
    """
    if seed:
        np.random.seed(seed)
    
    # partition data into positive and negative
    pos_sample = data.loc[data[target] == 1]
    neg_dat = data.loc[data[target] == 0]
    
    #choose negative sample to include
    neg_sample = neg_mul*len(pos_sample)
    negindex = np.random.choice(neg_dat.index, neg_sample, replace=0)
    
    #check that index aren't duplicated
    print(len(negindex) == len(set(negindex)))
    
    return pd.concat([pos_sample, data.loc[negindex,:]])
    
    

In [6]:
def SMOTE(data, target, size=1):
    """
    data = data you want to oversample
    """
    
    assert (0<size) & (size<=1)
    # partition data into positive and negative
    pos_sample = data.loc[data[target] == 1]
    neg_dat = data.loc[data[target] == 0]
    
    #choose positive sample to include
    posindex = np.random.choice(pos_sample.index, size*len(neg_dat), replace=1)
    pos_sample = data.loc[posindex, :]
    
    return pd.concat((neg_dat, pos_sample))

In [7]:
for i in tDat.columns:
    x = tDat[i].unique()
    print(i,':',len(x))

TransactionId : 95662
BatchId : 94809
AccountId : 3633
SubscriptionId : 3627
CustomerId : 3742
CurrencyCode : 1
CountryCode : 1
ProviderId : 6
ProductId : 23
ProductCategory : 9
ChannelId : 4
Amount : 1676
Value : 1517
TransactionStartTime : 94556
PricingStrategy : 4
FraudResult : 2


In [8]:
    
for i in tDat.columns:
    print('============================')
    print(i, ':')
    print('\n')
    print('mean:', round(tDat[i].value_counts().mean(), 2))
    print('std:', round(tDat[i].value_counts().std(), 2))
    print('min:', round(tDat[i].value_counts().min(),2))
    print('max:', round(tDat[i].value_counts().max(),2))
    print('25%:', round(tDat[i].value_counts().quantile(q=0.25),2))
    print('50%:', round(tDat[i].value_counts().quantile(q=0.5),2))
    print('75%:', round(tDat[i].value_counts().quantile(q=0.75),2))
    print('90% :', round(tDat[i].value_counts().quantile(q=0.9), 2))
    print('95% :', round(tDat[i].value_counts().quantile(q=0.95), 2))
    print('99% :', round(tDat[i].value_counts().quantile(q=0.99), 2))
    print('\n')


TransactionId :


mean: 1.0
std: 0.0
min: 1
max: 1
25%: 1.0
50%: 1.0
75%: 1.0
90% : 1.0
95% : 1.0
99% : 1.0


BatchId :


mean: 1.01
std: 0.17
min: 1
max: 28
25%: 1.0
50%: 1.0
75%: 1.0
90% : 1.0
95% : 1.0
99% : 1.0


AccountId :


mean: 26.33
std: 520.21
min: 1
max: 30893
25%: 2.0
50%: 4.0
75%: 14.0
90% : 38.0
95% : 66.0
99% : 189.68


SubscriptionId :


mean: 26.37
std: 548.33
min: 1
max: 32630
25%: 2.0
50%: 4.0
75%: 14.0
90% : 38.0
95% : 66.0
99% : 187.44


CustomerId :


mean: 25.56
std: 96.93
min: 1
max: 4091
25%: 2.0
50%: 7.0
75%: 20.0
90% : 57.9
95% : 98.0
99% : 275.95


CurrencyCode :


mean: 95662.0
std: nan
min: 95662
max: 95662
25%: 95662.0
50%: 95662.0
75%: 95662.0
90% : 95662.0
95% : 95662.0
99% : 95662.0


CountryCode :


mean: 95662.0
std: nan
min: 95662
max: 95662
25%: 95662.0
50%: 95662.0
75%: 95662.0
90% : 95662.0
95% : 95662.0
99% : 95662.0


ProviderId :


mean: 15943.67
std: 16460.28
min: 18
max: 38189
25%: 3723.75
50%: 10092.5
75%: 29275.0
90% : 36187.5
95% : 37188

TransactionId : super minimal change <br>
<b>BatchId</b> : all different <br>
AccountId: meaningful change <br>
SubscriptionId: meaningful change <br>
CustomerId: meaningful change <br>
<b>CurrencyCode </b>: all the same <br>
<b>CountryCode </b>: all the same <br>
ProviderId: meaningful change <br>
ProductId: meaningful change <br>
ProductCategory: meaningful change <br>
ChannelId : meaningful change <br>
Amount: numeric <br>
Value: numeric <br>
<b>TransactionStartTime </b>: all different <br>
PricingStrategy: meaningful change <br>

#### Data that is all the same going and all different going to be replaced by something more meaningful given that it gives significant signal for fraud

## detecting signal by feauture distribution

### BatchId

In [9]:
diff = tDat.groupby('BatchId')['BatchId'].count()[tDat.loc[tDat['FraudResult'] == 1,'BatchId'].unique()]

In [10]:
len(diff.loc[diff>1])/len(diff)

0.04918032786885246

In [11]:
counter = tDat.groupby('BatchId')['BatchId'].count()
print(len(counter.loc[counter>1])/len(counter))

0.006550011074897953


Customers that participate in more than one batch is less likely to commit fraud. This does make sense since you are less likely to do transaction in the same batch if you want to commit fraud. The feature doesn't said anything about fraud since most of the transaction is on one batch but it could be useful for some interaction later on.

### Currency Code and Country Code

Both of this feature is useless since it has zero variance so i will drop this

In [12]:
tDat = tDat.drop(['CurrencyCode','CountryCode'], axis=1)

### Datetime

In [13]:
#Converting to datetime
tDat.TransactionStartTime = pd.to_datetime(tDat.TransactionStartTime.apply(lambda x: x[:-1]))

In [14]:
bins = [0, 5, 13, 17, 25]
labels = ['Morning','Afternoon','Evening','Night']
hours = tDat['TransactionStartTime'].dt.hour
tDat['TimeCategory'] = pd.cut(hours-5+24 *(hours<5),bins=bins,labels=labels,right=False)

In [15]:
tDat['TimeCategory'].value_counts()/len(tDat)

Afternoon    0.509701
Morning      0.274832
Evening      0.166931
Night        0.048535
Name: TimeCategory, dtype: float64

In [16]:
tDat.loc[tDat['FraudResult'] ==1, 'TimeCategory'].value_counts()/len(tDat.loc[tDat['FraudResult'] ==1, 'TimeCategory'])

Afternoon    0.518135
Evening      0.212435
Morning      0.207254
Night        0.062176
Name: TimeCategory, dtype: float64

<b>It seems that time category has nothing to do with fraud</b>

In [17]:
tDat['DayName'] = tDat.TransactionStartTime.apply(lambda x: x.weekday_name)

/home/acer/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
  """Entry point for launching an IPython kernel.


In [18]:
round(tDat['DayName'].value_counts()/len(tDat), 2).sort_index()

Friday       0.29
Monday       0.12
Saturday     0.11
Sunday       0.10
Thursday     0.12
Tuesday      0.16
Wednesday    0.11
Name: DayName, dtype: float64

In [19]:
round(tDat.loc[tDat['FraudResult'] == 1,'DayName'].value_counts()/len(tDat.loc[tDat['FraudResult'] == 1]), 2).sort_index()

Friday       0.16
Monday       0.15
Saturday     0.13
Sunday       0.08
Thursday     0.18
Tuesday      0.16
Wednesday    0.15
Name: DayName, dtype: float64

Seems like fraud is excecuted in specific days. I wonder why though? next to test the significance i will perform statistical test

In [20]:
def prop_hypo_test(x, y, n1, n2, diff=0):
    import scipy.stats as st
    p1 = x/n1
    p2 = y/n2
    pgab = (x+y)/(n1+n2)
    sigma = (pgab*(1-pgab)*((1/n1)+(1/n2)))**0.5
    zvalue = (p1-p2-diff)/sigma
    print("your p value is : ", 1-st.norm.cdf(abs(zvalue)))

In [21]:
n1 = len(tDat)
n2 = len(tDat.loc[tDat['FraudResult'] == 1])
for k in tDat['DayName'].value_counts().index:
    print(k)
    x = tDat['DayName'].value_counts().loc[k]
    y = tDat.loc[tDat['FraudResult'] == 1, 'DayName'].value_counts().loc[k]
    prop_hypo_test(x, y, n1, n2)
    print("\n")

Friday
your p value is :  1.9824568163739897e-05


Tuesday
your p value is :  0.4498382046808508


Monday
your p value is :  0.1318682745659474


Thursday
your p value is :  0.004163466546359795


Wednesday
your p value is :  0.025968692532039928


Saturday
your p value is :  0.140063630314321


Sunday
your p value is :  0.13943610844419307




#### From the result above it seems that for some of the day, fraud transaction day is significantly different than usual transaction day. This will give good cue to the model

### Account Id

In [22]:
tDat.groupby('AccountId')['AccountId'].count().sort_values().describe()

count     3633.000000
mean        26.331407
std        520.214361
min          1.000000
25%          2.000000
50%          4.000000
75%         14.000000
max      30893.000000
Name: AccountId, dtype: float64

In [23]:
tDat.groupby('AccountId')['AccountId'].count().loc[tDat.loc[tDat['FraudResult'] == 1, 'AccountId'].to_list()].describe()

count      193.000000
mean       321.202073
std       2300.042675
min          1.000000
25%         15.000000
50%         50.000000
75%        106.000000
max      30893.000000
Name: AccountId, dtype: float64

From the above statistic we can conclude that account for fraud mostly done more than once transaction. To be honest this kind of odd since if we do a lot of transactions that means we are more visible. I thought fraud is done only using some unknown account. But if we look closly the amount of transaction is oddly big. This could signal they are actually hiding the fraud by other transaction thus it makes them harder to track.

In [24]:
tDat.loc[(tDat['AccountId'] == 'AccountId_4841') & (tDat['FraudResult'] == 1)]

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TimeCategory,DayName
68918,TransactionId_71725,BatchId_62457,AccountId_4841,SubscriptionId_3829,CustomerId_865,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-1005.0,1005,2019-01-23 09:15:37,2,1,Morning,Wednesday


Looking at the highest transaction frequency account the transaction that was clasified false is only once. This probably mean that the fraud actors borrow other people bank account and then used it for the fraud transaction. While, the account bearer receive some compesation for doing it.

In [25]:
tDat.loc[tDat['FraudResult'] == 1, 'AccountId'].value_counts()

AccountId_572     31
AccountId_4421    22
AccountId_1609    14
AccountId_4003    12
AccountId_1909     8
AccountId_1873     7
AccountId_4687     7
AccountId_4590     7
AccountId_2656     7
AccountId_1178     6
AccountId_3337     6
AccountId_1959     5
AccountId_4249     4
AccountId_523      4
AccountId_2366     3
AccountId_953      3
AccountId_3832     3
AccountId_4004     2
AccountId_4465     2
AccountId_1825     2
AccountId_830      2
AccountId_3943     2
AccountId_1488     2
AccountId_2043     2
AccountId_530      2
AccountId_1208     2
AccountId_2289     1
AccountId_1520     1
AccountId_352      1
AccountId_3519     1
AccountId_368      1
AccountId_1335     1
AccountId_1240     1
AccountId_3041     1
AccountId_4655     1
AccountId_3636     1
AccountId_1823     1
AccountId_777      1
AccountId_1480     1
AccountId_1754     1
AccountId_476      1
AccountId_3692     1
AccountId_3686     1
AccountId_1291     1
AccountId_2244     1
AccountId_1898     1
AccountId_1605     1
AccountId_463

### Subscription Id

In [53]:
not_unique = []
unique = []
for acc in tDat['AccountId'].unique():
    if len(tDat.loc[tDat['AccountId'] == acc, 'SubscriptionId'].unique()) != 1:
        print(len(tDat.loc[tDat['AccountId'] == acc, 'SubscriptionId'].unique()))
        not_unique.append(acc)
    else:
        unique.append(acc)
print(not_unique)
        
not_unique = []
unique = []

for acc in tDat['SubscriptionId'].unique():
    if len(tDat.loc[tDat['SubscriptionId'] == acc, 'AccountId'].unique()) != 1:
        print(len(tDat.loc[tDat['SubscriptionId'] == acc, 'AccountId'].unique()))
        not_unique.append(acc)
    else:
        unique.append(acc)
print(not_unique)

2
['AccountId_4841']
2
2
3
2
2
2
['SubscriptionId_3829', 'SubscriptionId_3489', 'SubscriptionId_4674', 'SubscriptionId_3495', 'SubscriptionId_4836', 'SubscriptionId_375']


only one account has more than one subscription id. And only some small number of subscription has more than one account. Nevertheless, it is still difference, maybe we can just ignore either accountid or subscription id. or make account that has more than one subscription to be reluctant to fraud and vice versa

In [58]:
tDat.loc[tDat['FraudResult'] == 1, 'SubscriptionId'].unique()

array(['SubscriptionId_4161', 'SubscriptionId_4834',
       'SubscriptionId_3451', 'SubscriptionId_1476',
       'SubscriptionId_3002', 'SubscriptionId_1417', 'SubscriptionId_666',
       'SubscriptionId_814', 'SubscriptionId_4174', 'SubscriptionId_4038',
       'SubscriptionId_539', 'SubscriptionId_3060', 'SubscriptionId_2964',
       'SubscriptionId_385', 'SubscriptionId_3597', 'SubscriptionId_4080',
       'SubscriptionId_2272', 'SubscriptionId_2562',
       'SubscriptionId_2044', 'SubscriptionId_4427',
       'SubscriptionId_1456', 'SubscriptionId_1252', 'SubscriptionId_695',
       'SubscriptionId_3303', 'SubscriptionId_3125', 'SubscriptionId_526',
       'SubscriptionId_3706', 'SubscriptionId_2550',
       'SubscriptionId_2416', 'SubscriptionId_162', 'SubscriptionId_726',
       'SubscriptionId_4429', 'SubscriptionId_1100', 'SubscriptionId_842',
       'SubscriptionId_2315', 'SubscriptionId_141', 'SubscriptionId_1355',
       'SubscriptionId_4345', 'SubscriptionId_2940', 'Subscri

When looking at the apperance of fraud account the account that has more than one subscription were found to be fraud. However, the accounts that have more than one subscription is only one so it is not enough to make conclusion. For SubscriptionId it seems that appearing in more than one account didn't affect fraud. So I will replace subscription Id Column with account having more than one subscription

### CustomerId

In [66]:
FraudCust = tDat.CustomerId.loc[tDat['FraudResult'] == 1].unique()

To be honest I dont know what is the difference between customer id and account id. So i will check to make sure it is different stuff.

In [79]:
not_unique = []
unique = []
for acc in tDat['CustomerId'].unique():
    if len(tDat.loc[tDat['CustomerId'] == acc, 'AccountId'].unique()) != 1:
        print(len(tDat.loc[tDat['CustomerId'] == acc, 'AccountId'].unique()))
        not_unique.append(acc)
    else:
        unique.append(acc)
print(not_unique)
        
not_unique = []
unique = []

for acc in tDat['AccountId'].unique():
    if len(tDat.loc[tDat['AccountId'] == acc, 'CustomerId'].unique()) != 1:
        print(len(tDat.loc[tDat['AccountId'] == acc, 'CustomerId'].unique()))
        not_unique.append(acc)
    else:
        unique.append(acc)
print(not_unique)

3
3
2
2
2
2
2
2
3
3
2
2
2
3
2
2
2
2
3
4
2
2
2
3
2
3
2
2
3
2
2
2
3
2
2
5
3
2
2
2
2
2
3
2
2
2
2
3
2
2
2
2
2
2
2
2
3
3
2
2
2
3
2
2
2
3
2
3
3
3
3
2
2
4
2
2
3
2
2
3
3
2
2
2
2
2
2
3
3
3
2
2
2
2
2
2
3
2
2
2
2
2
3
2
3
3
2
2
2
3
2
2
2
2
2
2
2
3
2
3
2
2
2
2
2
2
2
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
3
2
2
2
2
2
2
2
3
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
3
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
3
2
2
3
2
2
3
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
3
2
3
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
3
2
3
3
2
2
3
2
2
2
2
2
2
2
3
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
3
2
2
3
2
3
2
2
2
3
2
2
2
2
3
2
2
2
3
3
2
2
2
2
2
3
3
3
2
2
2
2
2
2
2
2
2
3
2
2
3
2
2
2
2
2
2
2
2
3
2
4
2
2
2
2
2
2
2
3
2
3
2
2
2
2


56
3
6
1723
182
14
20
11
7
['AccountId_4841', 'AccountId_4249', 'AccountId_168', 'AccountId_2366', 'AccountId_4840', 'AccountId_10', 'AccountId_398', 'AccountId_1074', 'AccountId_2420', 'AccountId_4713']


#### Okay both are different 

In [85]:
(tDat.CustomerId.value_counts()[FraudCust]>1).sum()/len(FraudCust)

0.8148148148148148

In [83]:
(tDat.CustomerId.value_counts()>1).sum()/len(tDat.CustomerId.value_counts())

0.8097274184927846

Kind of hard to conclude that this id is contributing to giving the signal. Because as you can see the percentage are roughly the same. So i think its better to leave them as it is and make them interaction features latter on. 

### Product Id and Category

In [87]:
(tDat.ProductId.value_counts())

ProductId_6     32635
ProductId_3     24344
ProductId_10    15384
ProductId_15    11964
ProductId_1      2885
ProductId_4      2403
ProductId_21     1512
ProductId_11     1377
ProductId_19      965
ProductId_14      789
ProductId_13      296
ProductId_2       236
ProductId_20      216
ProductId_8       197
ProductId_24      175
ProductId_22      112
ProductId_7        86
ProductId_27       31
ProductId_5        25
ProductId_9        17
ProductId_16       10
ProductId_12        2
ProductId_23        1
Name: ProductId, dtype: int64

In [93]:
FraudProd = tDat.loc[tDat['FraudResult'] == 1, 'ProductId'].unique()

In [96]:
(tDat.ProductId.value_counts()).loc[FraudProd].sort_values(ascending=False)

ProductId_6     32635
ProductId_3     24344
ProductId_10    15384
ProductId_15    11964
ProductId_21     1512
ProductId_13      296
ProductId_22      112
ProductId_5        25
ProductId_9        17
Name: ProductId, dtype: int64

Most of the product that have a tendency to be fraud is common product. It is also intresting to see which product categories do fraud transactions happen most

In [99]:
tDat.ProductCategory.value_counts()

financial_services    45405
airtime               45027
utility_bill           1920
data_bundles           1613
tv                     1279
ticket                  216
movies                  175
transport                25
other                     2
Name: ProductCategory, dtype: int64

In [101]:
tDat.loc[tDat['FraudResult'] == 1, 'ProductCategory'].unique()

array(['financial_services', 'airtime', 'utility_bill', 'transport'],
      dtype=object)

As expected we see financial services and transport which is susceptible to fraud transaction. The other is airtime and utility bill. I am not sure about this two since airtime products rarely packed into fraud product since its hard and for utility bill I am not sure what it is.